Тест различных типов нейросети для классификации писателей.

Каждый из нас писал в школе и университете изложения, сочинения, рефераты. А значит, в каждом из нас живет великий русский писатель.
В этой работе будем раскрывать свои таланты, находить себя в ряду таких гениев, как Пушкин, Гоголь, Грибоедов

В этой работе 
- скачаем корпус текстов 20-ми русских писателей. Каждый текст разобьем на обучающую и тестовую выборки.
- разработаем и обучим нейронную сеть определяющию авторство фрагментов текста (по тестовой выборке)
- скачаем СВОЕ сочинение (или чье-нибудь - есть в архиве). Сделаем из него проверочную выборку
- предложим нейронке предсказать автора сочинения (по проверочной выборке)
- объявим себя великим писателем, например, Гончаровым 

Ссылка на архив: https://storage.googleapis.com/datasets_ai/Data/writers25.zip

В работе рекомендуется пользоваться материалами из ноутбука практического занятия "Рекуррентные и одномерные сверточные нейронные сети". Допускается выбрать лучший вариант нейронки и адаптировать ее структуру, параметры обучения и формирования датасетов под свои *нужды*

In [ ]:
# ваше решение

In [ ]:
import gdown                                      # Подключим функцию gdown
file_id = '1TeJxgfr2ZEY5Elil7XgM8WW7njI9Toqj'     # Обозначим ID файла в рамках ссылки
file_name = 'writers25.zip'                         # Обозначим имя архива с базой
path = 'https://storage.googleapis.com/datasets_ai/Data/writers25.zip'  # Укажем директорию для скачивания: ссылка на гугл-диск + ID файла
gdown.download(path, file_name, quiet=False)      # Скачивание файла

Downloading...
From: https://storage.googleapis.com/datasets_ai/Data/writers25.zip
To: /content/writers25.zip
100%|██████████| 23.6M/23.6M [00:00<00:00, 190MB/s]


'writers25.zip'

In [ ]:
from itertools import chain
# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer
# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils
# Работа с массивами данных
import numpy as np 
# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential
# Основные слои
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation ,Embedding,SpatialDropout1D,Flatten,Conv1D,LSTM,MaxPooling1D
# Функции операционной системы
import os
# Регулярные выражения
import re

In [ ]:
# Распаковка архива в папку writers
!unzip -qo writers25.zip -d writers25/

In [ ]:
!ls writers25/rec_1d

refer  writters


In [ ]:
!ls writers25/rec_1d/writters

Беляев.txt    Гончаров.txt     Каверин.txt    Лесков.txt     Толстой.txt
Булгаков.txt  Горький.txt      Катаев.txt     Носов.txt      Тургенев.txt
Васильев.txt  Грибоедов.txt    Куприн.txt     Пастернак.txt  Чехов.txt
Гоголь.txt    Достоевский.txt  Лермонтов.txt  Пушкин.txt     Шолохов.txt


In [ ]:
!ls writers25/rec_1d/refer

островский.txt


In [ ]:
# Папка с текстовыми файлами
FILE_DIR  = 'writers25/rec_1d/writters/'

In [ ]:
os.listdir(FILE_DIR)

['Гончаров.txt',
 'Лесков.txt',
 'Чехов.txt',
 'Тургенев.txt',
 'Васильев.txt',
 'Горький.txt',
 'Пастернак.txt',
 'Достоевский.txt',
 'Беляев.txt',
 'Пушкин.txt',
 'Гоголь.txt',
 'Лермонтов.txt',
 'Каверин.txt',
 'Катаев.txt',
 'Носов.txt',
 'Булгаков.txt',
 'Куприн.txt',
 'Шолохов.txt',
 'Толстой.txt',
 'Грибоедов.txt']

In [ ]:
os.listdir('writers25/rec_1d/refer')

['островский.txt']

In [ ]:
import codecs
# В список добавляются имена классов
CLASS_LIST = []

# Здесь сохраняются тексты для обучения сети
text_train = []

# А здесь для проверки точности сети
text_test = []

TRAIN_RATIO=0.9

# цикл для итерации по каждому имени текста в общей папке
for file_name in os.listdir(FILE_DIR):

    # Выделение имени класса и типа выборки из имени файла
    m = re.match('(.+)(\.txt)', file_name)# (\S+)_
    class_name=m[1]
    # Если выделение получилось, то файл обрабатывается
    if m:

            # Добавление нового класса, если его еще нет в списке
        if class_name not in CLASS_LIST:

                # Выводится информационное сообщение о добавлении названия класса
                CLASS_LIST.append(class_name)

                # Инициализация соответствующих классу строк текста
                text_train.append('')
                text_test.append('')

            # Поиск индекса класса для добавления содержимого файла в выборку
        cls = CLASS_LIST.index(class_name)

            # Выводится информационное сообщение о добавлении класса в список классов и текста к выборке
        print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}"')

            # оператор with - безопасное чтение каждого файла с текстом
        with codecs.open(f'{FILE_DIR}/{file_name}', 'r','cp1251') as f:

                # Загрузка содержимого файла в строку
                text = f.read()
                #b=bytes(text,"cp1251")
                #text=str(b,"cp866")
                #text.decode('utf8').encode('cp866')
                text = text.replace('\n', ' ')

            # Добавление текста к соответствующей выборке класса. Концы строк заменяются на пробел

        text_train[cls] += ' ' + text[:int(len(text)*TRAIN_RATIO)]
        text_test[cls] += ' ' + text[int(len(text)*TRAIN_RATIO):]
        
            
# Определим кол-во классов
CLASS_COUNT = len(CLASS_LIST)

Добавление файла "Гончаров.txt" в класс "Гончаров"
Добавление файла "Лесков.txt" в класс "Лесков"
Добавление файла "Чехов.txt" в класс "Чехов"
Добавление файла "Тургенев.txt" в класс "Тургенев"
Добавление файла "Васильев.txt" в класс "Васильев"
Добавление файла "Горький.txt" в класс "Горький"
Добавление файла "Пастернак.txt" в класс "Пастернак"
Добавление файла "Достоевский.txt" в класс "Достоевский"
Добавление файла "Беляев.txt" в класс "Беляев"
Добавление файла "Пушкин.txt" в класс "Пушкин"
Добавление файла "Гоголь.txt" в класс "Гоголь"
Добавление файла "Лермонтов.txt" в класс "Лермонтов"
Добавление файла "Каверин.txt" в класс "Каверин"
Добавление файла "Катаев.txt" в класс "Катаев"
Добавление файла "Носов.txt" в класс "Носов"
Добавление файла "Булгаков.txt" в класс "Булгаков"
Добавление файла "Куприн.txt" в класс "Куприн"
Добавление файла "Шолохов.txt" в класс "Шолохов"
Добавление файла "Толстой.txt" в класс "Толстой"
Добавление файла "Грибоедов.txt" в класс "Грибоедов"


In [ ]:
print(len(text_train))
print(len(text_train[0]))
print(len(text_test))
print(len(text_test[0]))
print(CLASS_LIST)

20
2819197
20
313246
['Гончаров', 'Лесков', 'Чехов', 'Тургенев', 'Васильев', 'Горький', 'Пастернак', 'Достоевский', 'Беляев', 'Пушкин', 'Гоголь', 'Лермонтов', 'Каверин', 'Катаев', 'Носов', 'Булгаков', 'Куприн', 'Шолохов', 'Толстой', 'Грибоедов']


In [ ]:
print(text_train[0][:100])

 В Гороховой улице, в одном из больших домов, народонаселения которого ста


Токенизируем

In [ ]:
# Объем словаря для токенизатора
VOCAB_SIZE = 20000

# Длина отрезка текста (окна) в словах                        
WIN_SIZE   = 2000

# Шаг окна разбиения текста на векторы                         
WIN_HOP    = 1000                          

# Токенайзер из Keras для разбиения текста и построения частотного словаря
tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', 
                          lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

In [ ]:
# Получаем словарь частотности 
tokenizer.fit_on_texts(text_train)

In [ ]:
seq_train = tokenizer.texts_to_sequences(text_train)
seq_test = tokenizer.texts_to_sequences(text_test)

In [ ]:
print(len(text_train[0]))
print(len(seq_train[0]))
print()
SequencyLen=[]
SequencyLenMax=0
SequencyLenMin=1000000000
for i,seq in enumerate(seq_train):
  if(len(seq)>SequencyLenMax):SequencyLenMax=len(seq)
  if(len(seq)<SequencyLenMin):SequencyLenMin=len(seq)
  SequencyLen.append(len(seq))
  print(i,len(seq))
print()
print("Max len=",SequencyLenMax,"Min len=",SequencyLenMin,"Max/Min=",SequencyLenMax/SequencyLenMin)

2819197
468373

0 468373
1 320444
2 1001500
3 291890
4 438541
5 382624
6 439265
7 766007
8 329700
9 477414
10 285976
11 294132
12 292981
13 364563
14 369728
15 295908
16 336979
17 476043
18 491711
19 147020

Max len= 1001500 Min len= 147020 Max/Min= 6.811998367569038


Балансировка класов путем простого дублирования записей

In [ ]:
#Балансировка

In [ ]:
seq_train_balanced=[]
for seq in seq_train:
  ptr=0
  CurrentLen=len(seq)
  temp_seq=np.array(seq)
  while(len(temp_seq)<SequencyLenMax):
    AddEntries=SequencyLenMax-len(temp_seq)
    if(AddEntries>len(temp_seq)):
          AddEntries=np.size(temp_seq)
    temp_seq=np.hstack((temp_seq,temp_seq[0:AddEntries]))
  seq_train_balanced.append(temp_seq)

Проверка балансировки

In [ ]:
for i,seq in enumerate(seq_train_balanced):
  print(i,len(seq))

0 1001500
1 1001500
2 1001500
3 1001500
4 1001500
5 1001500
6 1001500
7 1001500
8 1001500
9 1001500
10 1001500
11 1001500
12 1001500
13 1001500
14 1001500
15 1001500
16 1001500
17 1001500
18 1001500
19 1001500


In [ ]:
def split_sequence(sequence,   # Последовательность индексов
                   win_size,   # Размер окна для деления на примеры
                   hop):       # Шаг окна

    # Последовательность разбивается на части до последнего полного окна
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

In [ ]:
def vectorize_sequence(seq_list,    # Список последовательностей индексов 
                       win_size,    # Размер окна для деления на примеры
                       hop):        # Шаг окна

    # В списке последовательности следуют в порядке их классов (их кол-во сповпадает с кол-вом классов)
    class_count = len(seq_list)

    # Списки для исходных векторов и категориальных меток класса
    x, y = [], []

    # Для каждого класса:
    for cls in range(class_count):

        # Разбиение последовательности класса cls на отрезки
        vectors = split_sequence(seq_list[cls], win_size, hop)

        # Добавление отрезков в выборку

        x += vectors
        
        # Для всех отрезков класса cls добавление меток класса в виде OHE
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    # Возврат результатов как numpy-массивов
    return np.array(x), np.array(y)

In [ ]:
# Формирование обучающей выборки
x_train, y_train = vectorize_sequence(seq_train_balanced, WIN_SIZE, WIN_HOP) 
# Формирование тестовой выборки
x_test, y_test = vectorize_sequence(seq_test, WIN_SIZE, WIN_HOP)

# Проверка формы сформированных данных
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(20000, 2000) (20000, 20)
(894, 2000) (894, 20)


In [ ]:
x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())#
x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())#
print(x_train_01.shape,x_test_01.shape)

(20000, 20000) (894, 20000)


Полносвязная модель с BOW

In [ ]:
model_BoW = Sequential()                                            # Создание последовательной модели нейросети
model_BoW.add(Dense(100, input_dim=x_train_01.shape[-1], activation="relu"))  # Первый полносвязный слой
model_BoW.add(BatchNormalization())
model_BoW.add(Dropout(0.5))
model_BoW.add(Dense(50, activation="relu"))                         # Второй полносвязный слой
model_BoW.add(BatchNormalization())
model_BoW.add(Dropout(0.5))
model_BoW.add(Dense(33, activation="relu"))                         # Третий полносвязный слой
model_BoW.add(BatchNormalization())                                 # Слой пакетной нормализации
model_BoW.add(Dropout(0.1))                                         # Слой регуляризации Dropout
model_BoW.add(Dense(y_train.shape[-1], activation='softmax'))             # Выходной полносвязный слой
model_BoW.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model_BoW.fit(x_train_01,y_train,epochs=20,batch_size=32,validation_data=(x_test_01, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/20
586/586 [==============================] - 12s 19ms/step - loss: 1.3815 - accuracy: 0.6166 - val_loss: 0.3793 - val_accuracy: 0.9344
Epoch 2/20
586/586 [==============================] - 11s 19ms/step - loss: 0.3685 - accuracy: 0.9018 - val_loss: 0.2377 - val_accuracy: 0.9392
Epoch 3/20
586/586 [==============================] - 11s 18ms/step - loss: 0.1934 - accuracy: 0.9473 - val_loss: 0.2357 - val_accuracy: 0.9523
Epoch 4/20
586/586 [==============================] - 11s 18ms/step - loss: 0.1485 - accuracy: 0.9569 - val_loss: 0.2414 - val_accuracy: 0.9523
Epoch 5/20
586/586 [==============================] - 11s 18ms/step - loss: 0.1207 - accuracy: 0.9642 - val_loss: 0.1969 - val_accuracy: 0.9535
Epoch 6/20
586/586 [==============================] - 11s 18ms/step - loss: 0.1079 - accuracy: 0.9670 - val_loss: 0.2390 - val_accuracy: 0.9511
Epoch 7/20
586/586 [==============================] - 11s 18ms/step - loss: 0.1003 - accuracy: 0.9684 - val_loss: 0.2739 - val_accuracy:

Полносвязная модель с эмбедингом

In [ ]:
model_text_emb_20 = Sequential()                                            # Создание последовательной модели нейросети
model_text_emb_20.add(Embedding(VOCAB_SIZE, 20, input_length=x_train.shape[-1]))
model_text_emb_20.add(SpatialDropout1D(0.2))
model_text_emb_20.add(Flatten())
model_text_emb_20.add(Dense(100, activation="relu"))  # Первый полносвязный слой
model_text_emb_20.add(BatchNormalization())
model_text_emb_20.add(Dropout(0.5))
model_text_emb_20.add(Dense(50, activation="relu"))                         # Второй полносвязный слой
model_text_emb_20.add(BatchNormalization())
model_text_emb_20.add(Dropout(0.5))
model_text_emb_20.add(Dense(33, activation="relu"))                         # Третий полносвязный слой
model_text_emb_20.add(BatchNormalization())                                 # Слой пакетной нормализации
model_text_emb_20.add(Dropout(0.1))                                         # Слой регуляризации Dropout
model_text_emb_20.add(Dense(y_train.shape[-1], activation='softmax'))             # Выходной полносвязный слой
model_text_emb_20.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model_text_emb_20.fit(x_train,y_train,epochs=20,batch_size=32,validation_data=(x_test, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/20
625/625 [==============================] - 33s 50ms/step - loss: 2.9394 - accuracy: 0.1122 - val_loss: 2.3551 - val_accuracy: 0.2640
Epoch 2/20
625/625 [==============================] - 32s 50ms/step - loss: 1.9503 - accuracy: 0.3349 - val_loss: 1.7411 - val_accuracy: 0.3915
Epoch 3/20
625/625 [==============================] - 32s 51ms/step - loss: 1.2548 - accuracy: 0.5609 - val_loss: 1.2755 - val_accuracy: 0.5805
Epoch 4/20
625/625 [==============================] - 32s 51ms/step - loss: 0.8221 - accuracy: 0.7171 - val_loss: 0.9047 - val_accuracy: 0.7282
Epoch 5/20
625/625 [==============================] - 32s 51ms/step - loss: 0.5754 - accuracy: 0.8041 - val_loss: 0.7674 - val_accuracy: 0.7718
Epoch 6/20
625/625 [==============================] - 31s 50ms/step - loss: 0.4366 - accuracy: 0.8523 - val_loss: 1.0328 - val_accuracy: 0.7002
Epoch 7/20
625/625 [==============================] - 31s 50ms/step - loss: 0.3434 - accuracy: 0.8860 - val_loss: 0.9014 - val_accuracy:

Сверточная 2х слойная с эмбедингом

In [ ]:
model_LC_1 = Sequential()
model_LC_1.add(Embedding(VOCAB_SIZE, 20, input_length=WIN_SIZE))
#model_LC_3.add(SpatialDropout1D(0.2))
# Сверточный слой
model_LC_1.add(Conv1D(20, 5, activation='relu'))
model_LC_1.add(Conv1D(40, 5, activation='relu'))
# Слой регуляризации Dropout
model_LC_1.add(Dropout(0.5))
# Слой пакетной нормализации
#model_LC_3.add(BatchNormalization())
# Рекуррентный слой LSTM
#model_LC_3.add(LSTM(4, return_sequences=1))
#model_LC_1.add(Dense(33, activation="relu")) 
#model_LC_3.add(Dropout(0.2))
#model_LC_3.add(BatchNormalization())
# Слой выравнивания/сглаживания ("сплющивание" данных в вектор)
model_LC_1.add(Flatten())
model_LC_1.add(Dense(CLASS_COUNT, activation='softmax'))
model_LC_1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model_LC_1.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/10
625/625 [==============================] - 110s 175ms/step - loss: 1.8296 - accuracy: 0.3808 - val_loss: 1.7688 - val_accuracy: 0.5179
Epoch 2/10
625/625 [==============================] - 111s 178ms/step - loss: 0.1688 - accuracy: 0.9439 - val_loss: 1.9487 - val_accuracy: 0.6040
Epoch 3/10
625/625 [==============================] - 109s 174ms/step - loss: 0.0218 - accuracy: 0.9937 - val_loss: 2.1167 - val_accuracy: 0.6051
Epoch 4/10
625/625 [==============================] - 109s 174ms/step - loss: 0.0253 - accuracy: 0.9912 - val_loss: 3.0602 - val_accuracy: 0.5626
Epoch 5/10
625/625 [==============================] - 109s 175ms/step - loss: 0.0302 - accuracy: 0.9894 - val_loss: 3.1902 - val_accuracy: 0.5481
Epoch 6/10
625/625 [==============================] - 110s 175ms/step - loss: 0.0213 - accuracy: 0.9926 - val_loss: 2.9495 - val_accuracy: 0.6119
Epoch 7/10
625/625 [==============================] - 109s 174ms/step - loss: 0.0114 - accuracy: 0.9961 - val_loss: 3.7930 -

Сверточная 1 слойная с эмбедингом

In [ ]:
model_LC_3 = Sequential()
model_LC_3.add(Embedding(VOCAB_SIZE, 20, input_length=WIN_SIZE))
model_LC_3.add(SpatialDropout1D(0.2))
# Сверточный слой
model_LC_3.add(Conv1D(20, 5, activation='relu'))
# Слой подвыборки/пулинга с функцией максимума
# model_LC_3.add(MaxPooling1D(2))
# Слой регуляризации Dropout
model_LC_3.add(Dropout(0.5))
# Слой пакетной нормализации
#model_LC_3.add(BatchNormalization())
# Рекуррентный слой LSTM
#model_LC_3.add(LSTM(4, return_sequences=1))
#model_LC_3.add(Dense(33, activation="relu")) 
#model_LC_3.add(Dropout(0.2))
#model_LC_3.add(BatchNormalization())
# Слой выравнивания/сглаживания ("сплющивание" данных в вектор)
model_LC_3.add(Flatten())
model_LC_3.add(Dense(CLASS_COUNT, activation='softmax'))
model_LC_3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model_LC_3.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/10
625/625 [==============================] - 114s 89ms/step - loss: 1.7755 - accuracy: 0.4171 - val_loss: 1.2609 - val_accuracy: 0.6253
Epoch 2/10
625/625 [==============================] - 56s 90ms/step - loss: 0.1312 - accuracy: 0.9621 - val_loss: 0.9852 - val_accuracy: 0.7349
Epoch 3/10
625/625 [==============================] - 56s 89ms/step - loss: 0.0312 - accuracy: 0.9930 - val_loss: 1.1501 - val_accuracy: 0.7662
Epoch 4/10
625/625 [==============================] - 56s 89ms/step - loss: 0.0192 - accuracy: 0.9941 - val_loss: 1.1700 - val_accuracy: 0.7651
Epoch 5/10
625/625 [==============================] - 56s 90ms/step - loss: 0.0208 - accuracy: 0.9932 - val_loss: 1.1828 - val_accuracy: 0.7550
Epoch 6/10
625/625 [==============================] - 56s 90ms/step - loss: 0.0222 - accuracy: 0.9930 - val_loss: 1.5242 - val_accuracy: 0.7148
Epoch 7/10
625/625 [==============================] - 56s 89ms/step - loss: 0.0193 - accuracy: 0.9930 - val_loss: 1.5864 - val_accuracy

Обработка файла сочинения

In [ ]:
text_val_list=[]
with codecs.open('writers25/rec_1d/refer/островский.txt', 'r','cp1251') as f:
  text_val = f.read()
  text_val = text.replace('\n', ' ')
  text_val_list.append(text_val)

In [ ]:
print(text_val[:10],type(text_val))

ГОРЕ ОТ УМ <class 'str'>


In [ ]:
seq_val = tokenizer.texts_to_sequences(text_val_list)

In [ ]:
print(np.shape(seq_val[0]),len(seq_val[0]),type(seq_val[0]),seq_val[0][:100])
print(np.shape(seq_test[0]),len(seq_test[0]),type(seq_test[0]),seq_test[0][:100])

(162402,) 162402 <class 'list'> [952, 26, 601, 2, 17584, 2, 1, 497, 17475, 1, 8490, 4, 1, 385, 2, 12305, 1363, 724, 14, 2, 533, 18559, 2, 699, 1940, 18289, 2008, 1, 1, 20, 52, 4, 468, 2, 377, 1444, 11510, 2, 1128, 1, 1078, 9438, 2, 1243, 2669, 17661, 1, 1, 2, 6857, 5067, 643, 1, 203, 1, 3, 1048, 371, 14, 8, 1, 1, 2, 1541, 1, 1, 1541, 1, 3031, 7302, 1, 2, 795, 1, 1, 1, 2, 444, 507, 2, 444, 1121, 2, 1, 2, 6708, 3, 138, 1, 7873, 2, 1448, 1137, 833, 1, 3, 48, 9266, 106, 1]
(50655,) 50655 <class 'list'> [3, 1529, 97, 4599, 5, 625, 3, 38, 1, 129, 12406, 1, 97, 18, 1, 150, 6723, 10, 1, 8, 766, 22, 18373, 3, 239, 3496, 1, 5741, 2, 2, 16, 17, 5, 24, 631, 27, 367, 55, 40, 5817, 8, 52, 7768, 3, 1, 23, 15618, 1, 1, 1, 8, 1, 1, 14465, 12063, 31, 4417, 1, 1, 226, 3, 531, 1, 3077, 8850, 3, 78, 1, 15, 1915, 236, 5, 6870, 1, 17, 27, 1017, 1, 1116, 515, 21, 426, 6637, 39, 97, 1, 97, 1, 12884, 1, 199, 1590, 19, 1, 3, 10085, 21, 8070, 35, 19]


In [ ]:
x_val=[seq_val[0][i:i + WIN_SIZE] for i in range(0, len(seq_val[0]) - WIN_SIZE + 1, WIN_HOP)]
#x_val=np.array(x_val)
print(np.shape(x_val))

(161, 2000)


In [ ]:
print(x_val[0:3][:10])

[[952, 26, 601, 2, 17584, 2, 1, 497, 17475, 1, 8490, 4, 1, 385, 2, 12305, 1363, 724, 14, 2, 533, 18559, 2, 699, 1940, 18289, 2008, 1, 1, 20, 52, 4, 468, 2, 377, 1444, 11510, 2, 1128, 1, 1078, 9438, 2, 1243, 2669, 17661, 1, 1, 2, 6857, 5067, 643, 1, 203, 1, 3, 1048, 371, 14, 8, 1, 1, 2, 1541, 1, 1, 1541, 1, 3031, 7302, 1, 2, 795, 1, 1, 1, 2, 444, 507, 2, 444, 1121, 2, 1, 2, 6708, 3, 138, 1, 7873, 2, 1448, 1137, 833, 1, 3, 48, 9266, 106, 1, 2, 1, 1, 2, 2786, 4, 827, 4, 468, 1, 2, 2786, 1, 1797, 5723, 1, 1, 4, 101, 942, 1, 3216, 228, 4, 3561, 1, 1, 1058, 7473, 8, 1, 143, 76, 1, 16303, 1, 504, 1597, 1, 8, 9136, 810, 253, 123, 1, 2, 533, 90, 14334, 6573, 8, 9136, 15047, 2, 1, 219, 12, 257, 260, 1, 2, 481, 1, 511, 10658, 2, 16134, 378, 1727, 315, 32, 315, 2, 5, 5778, 1, 5, 1, 70, 2379, 2, 55, 43, 36, 7, 1, 2, 92, 123, 146, 126, 2, 1, 15, 1, 2, 437, 2, 1484, 2596, 1363, 1393, 2, 10741, 7144, 875, 19, 260, 2, 29, 1, 699, 10249, 2, 2117, 3, 1246, 48, 5, 1614, 2, 11642, 26, 1506, 2, 51, 1655, 1,

In [ ]:
print(x_val[0][:30])

[952, 26, 601, 2, 17584, 2, 1, 497, 17475, 1, 8490, 4, 1, 385, 2, 12305, 1363, 724, 14, 2, 533, 18559, 2, 699, 1940, 18289, 2008, 1, 1, 20]


In [ ]:
Predict=model_LC_3.predict(x_val)

In [ ]:
PredictClass=np.argmax(Predict,axis=1)

In [ ]:
print(np.shape(Predict))

(161, 20)


In [ ]:
np.bincount(PredictClass)

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   1,   0,   0, 160])

In [ ]:
print([CLASS_LIST[i] for i in PredictClass])

['Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов', 'Грибоедов'

In [ ]:
#160 кусков сочинения из 161 определяет как Грибоедов

Вывод по BOW классифицировать текст проще всего, т.к. писателя легко определить по набору используемых слов.